# BT01: Ẩn tin mật trên văn bản bằng phương pháp chèn khoảng trắng <font color="grey">(chèn khoảng trắng giữa các từ để làm cho văn bản được căn lề 2 bên)


Họ tên: Vũ Mạnh Hùng

MSSV: 18424029

---
## Cách làm bài và nộp bài
&#9889; Bạn lưu ý là mình sẽ dùng chương trình hỗ trợ chấm bài nên bạn cần phải tuân thủ chính xác qui định mà mình đặt ra, nếu không rõ thì hỏi, chứ không nên tự tiện làm theo ý của cá nhân.

**Cách làm bài**

Bạn sẽ làm trực tiếp trên file notebook này. Đầu tiên, bạn điền họ tên và MSSV vào phần đầu file ở bên trên. Trong file, bạn làm bài ở những chỗ có ghi là:
```python
# YOUR CODE HERE
raise NotImplementedError()
```
hoặc đối với những phần code không bắt buộc thì là:
```python
# YOUR CODE HERE (OPTION)
```
hoặc đối với markdown cell thì là:
```markdown
YOUR ANSWER HERE
```
Tất nhiên, khi làm thì bạn xóa dòng `raise NotImplementedError()` đi.
Đối những phần yêu cầu code thì thường ở ngay phía dưới sẽ có một (hoặc một số) cell chứa các bộ test để giúp bạn biết đã code đúng hay chưa; nếu chạy cell này không có lỗi gì thì có nghĩa là qua được các bộ test. Trong một số trường hợp, các bộ test có thể sẽ không đầy đủ; nghĩa là, nếu không qua được test thì là code sai, nhưng nếu qua được test thì chưa chắc đã đúng hoàn toàn.

Trong khi làm bài, bạn có thể cho in ra màn hình, tạo thêm các cell để test. Nhưng khi nộp bài thì bạn xóa các cell mà bạn tự tạo, xóa hoặc comment các câu lệnh in ra màn hình. Bạn lưu ý <font color=red>không được tự tiện xóa các cell hay sửa code của Thầy</font> (trừ những chỗ được phép sửa như đã nói ở trên).

Trong khi làm bài, thường xuyên `Ctrl + S` để lưu lại bài làm của bạn, tránh mất mát thông tin.


*Nên nhớ mục tiêu chính ở đây là <font color=green>học, học một cách chân thật</font>. Bạn có thể thảo luận ý tưởng với bạn khác, nhưng <font color=green>code và bài làm phải là của bạn, dựa trên sự hiểu thật sự của bạn</font>. <font color=red>Nếu vi phạm thì sẽ bị 0 điểm cho toàn bộ môn học.</font>*

**Cách nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Kernel` - `Restart Kernel & Run All Cells`, để restart và chạy tất cả các cell trong notebook của bạn; do đó, trước khi nộp bài, bạn nên chạy thử `Kernel` - `Restart Kernel & Run All Cells` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, bạn tạo thư mục nộp bài theo cấu trúc sau:
- Thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`)
    - Thư mục `BT01`
        - File `BT01-AnTinMatTrenVanBan_KhoangTrang.ipynb` (không cần nộp các file khác)

Cuối cùng, bạn nén thư mục `MSSV` này lại và nộp ở link trên moodle. <font color=red>Bạn lưu ý tuân thủ chính xác cấu trúc này.</font>

## 2. Import

In [2]:
import bitarray

In [2]:
'''
Lấy danh sách các dòng không nhúng được
Các tham số:
    cover_text_lines(str): Nội dung văn bản phủ
'''
def getAllLineNoneAlignment(cover_text_lines):
    rs = []

    for idx in range(len(cover_text_lines)):
        # Những dòng không phải dòng căn lề (dòng cuối đoạn văn và dòng không có kí tự)
        if((idx + 1 < len(cover_text_lines) and not cover_text_lines[idx + 1]) or idx == len(cover_text_lines)):
            rs.append(idx)
            rs.append(idx + 1)
        
        # Dòng cuối cùng của văn bản phủ
        if(idx + 1 == len(cover_text_lines)):
            rs.append(idx)              
    return rs

In [3]:
'''
Tìm kiếm tất cả các khoảng trắng của dòng
Các tham số:
    text_line(str): Dòng dữ liệu
'''
def findAllSpaceInLine(text_line):
    rs = []
    idx = 0
    while idx < len(text_line):
        idx = text_line.find(' ', idx)
        if idx == -1:
            break
        rs.append(idx)
        idx += 1
    return rs

In [4]:
'''
Lấy index các khoảng trắng chèn thêm khoảng trắng
Các tham số:
    line (str): Dòng dữ liệu
    msg_bits (str): Dãy bit cần nhúng
    idx_bit (int): Index của bit cần nhúng đầu tiên
    lst_space (list): Danh sách index các khoảng trắng của dòng
    text_width (int): Số ký tự tối đa của một dòng dữ liệu
'''
def getAllSpaceInsert(line, msg_bits, idx_bit, lst_space, text_width):
    n = len(lst_space)                              # Số khoảng trắng giữa các dòng
    m = text_width - len(line)                      # Số khoảng trắng chèn thêm được, số khoảng trắng căn lề (Đếm số lượng các khoảng trắng cuối dòng)
    numBitEmbed = min(m, n - m)                     # Số bit có thể nhúng
    # -----------------------------------------------------    

    rs = []
    idxInsert = -1                          # Index của khoảng trắng sẽ chèn thêm 1 khoảng trắng
    idxTemp = -1                            # Index kiểm tra nhúng
    count = 0                               # Đếm số lần nhúng
    # -----------------------------------------------------    

    # Xác định những vị trí cần chèn khoảng trắng
    for idx in range(len(lst_space)):
        if(idx == idxTemp):                 # Nếu khoảng trắng đang xét thuộc cặp khoảng trắng đã nhúng thì xét khoảng trắng tiếp theo
            continue
        
        if(count >= numBitEmbed):           # Nếu nhúng đủ bit thì xét
            if(numBitEmbed == (n - m)):     # Nếu vẫn cần chèn khoảng trắng để đủ khoảng trắng căn lề
                temp = m - numBitEmbed
                for item in lst_space[idx : idx + temp]:
                    rs.append(item)
            break        
        
        if(msg_bits[idx_bit]):     # Nhúng bit 1
            idxInsert = lst_space[idx + 1]
        else:                      # Nhúng bit 0
            idxInsert = lst_space[idx]

        rs.append(idxInsert)
        idxTemp = idx + 1
        idx_bit += 1
        count += 1
    return rs

## 3. Hàm nhúng (6đ)

In [5]:
def embed(msg_file, cover_text_file, text_width, stego_text_file):
    '''
    Nhúng tin mật vào văn bản bằng phương pháp chèn khoảng trắng 
    để văn bản được căn lề 2 bên.
    
    Các tham số:
        msg_file (str): Tên file chứa secret message.
        cover_text_file (str): Tên file chứa cover text.
        text_width (int): Chiều dài của dòng sau khi được căn lề.
        stego_text_file (str): Tên file chứa stego text.
    Giá trị trả về:
        bool: True nếu nhúng thành không, False nếu không đủ chỗ để nhúng. 
    '''
    # Đọc file văn bản phủ
    with open(cover_text_file, 'r') as fcover:
        cover_text_lines = fcover.read().splitlines()

    # Đọc file tin mật
    with open(msg_file, 'r') as f:
        msg = f.read()

    # Chuyển đổi tin mật sang dạng tin mật
    msg_bits = bitarray.bitarray()
    msg_bits.fromstring(msg)
    
    # Lấy danh sách index các dòng không nhúng được
    lstNoneAlign = getAllLineNoneAlignment(cover_text_lines)

    #===================================================================================================
    # Tiến hành nhúng tin mật
    stego_text = ''
    idxBitEmbed = 0
    flag = False

    for idxLine, line in enumerate(cover_text_lines):       # Duyệt từng dòng dữ liệu của văn bản phủ
        if(idxLine in lstNoneAlign):                        # Nếu dòng dữ liệu không phải dòng căn lề thì xét đến dòng tiếp theo
            stego_text += line
            if(line != cover_text_lines[-1]):
                stego_text += '\n'            
            continue

        lstSpace = findAllSpaceInLine(line)             # Lấy danh sách các khoảng trắng của dòng
        n = len(lstSpace)                               # Số khoảng trắng giữa các dòng
        m = text_width - len(line)                      # Số khoảng trắng chèn thêm được, số khoảng trắng căn lề (Đếm số lượng các khoảng trắng cuối dòng)
        #------------------------------------------------------------------------------------------------            

        # TH: Không cần khoảng trắng chèn để căn lề  5 12 28 32 36 48 53
        if(m <= 0):
            stego_text += line + '\n'
            continue
        #------------------------------------------------------------------------------------------------            

        # TH: Không nhúng được nhưng vẫn cần chèn khoảng trắng để căn lề: 5 17 55
        if(m >= n):
            # Chèn một khoảng trắng vào mỗi khoảng trắng trong dòng
            for idxInsert, item in enumerate(lstSpace):
                idxSpace = item + idxInsert
                line = line[:idxSpace] + ' ' + line[idxSpace:]

            if(m > n):
                countInsert = n
                for idxInsert, item in enumerate(lstSpace):
                    if(countInsert == m):
                        break

                    idxSpace = item + idxInsert + 1
                    line = line[:idxSpace] + ' ' + line[idxSpace:]
                    countInsert += 1

            stego_text += line + '\n'
            continue
        #------------------------------------------------------------------------------------------------            

        numBitEmbed = min(m, n - m)                      # Số bit có thể nhúng

        # Xử lý khi hết bit nhúng        
        if(idxBitEmbed  + numBitEmbed > len(msg_bits)):
            numBitAdd =  numBitEmbed
            if(flag):
                bit_add = [False] * numBitAdd
                msg_bits.extend(bit_add)                # Thêm  một list bit 0 vào
            else:
                msg_bits.append(True)                   # Thêm bit 1 vào  
                bit_add = [False] * (numBitAdd - 1) 
                msg_bits.extend(bit_add)                # Thêm  một list bit 0 vào
                flag = True
        #------------------------------------------------------------------------------------------------            
        
        lstSpaceInsert = getAllSpaceInsert(line, msg_bits, idxBitEmbed, lstSpace, text_width)
        idxBitEmbed += numBitEmbed
        
        for idxInsert, item in enumerate(lstSpaceInsert):
            idxSpace = item + idxInsert
            line = line[:idxSpace] + ' ' + line[idxSpace:]

        stego_text += line + '\n'
    #===================================================================================================

    if(not flag):
        print('Embed Fail!')
        return False       

    if(idxBitEmbed < len(msg_bits)):
        print('Not enough spaces to embed!')
        return False

    # Ghi kết ra file stego.txt
    with open(stego_text_file, 'w') as f:
        f.write(stego_text)
    return True

In [6]:
# TEST 1            
result = embed('msg.txt', 'cover.txt', 70, 'stego.txt')
assert result == True
with open('stego.txt', 'r') as f:
    stego_text = f.read()
with open('correct_stego.txt', 'r') as f:
    correct_stego_text = f.read()
assert stego_text == correct_stego_text

In [7]:
# TEST 2
result = embed('msg2.txt', 'cover.txt', 70, 'stego.txt')
assert result == False

Embed Fail!


## 4. Hàm rút trích (4đ)

In [8]:
def extract(stego_text_file, extr_msg_file):
    '''
    Hàm rút trích tin mật đã được nhúng bằng phương pháp chèn khoảng trắng.
    
    Các tham số:
        stego_text_file (str): Tên file chứa stego text.
        extr_msg_file (str): Tên file chứa secret message được rút trích.
    '''
    # Đọc file văn bản đã nhúng tin mật
    with open(stego_text_file, 'r') as fstego:
        cover_text_lines = fstego.read().splitlines()
        
    # Lấy danh sách index các dòng không nhúng được
    lstNoneAlign = getAllLineNoneAlignment(cover_text_lines)
    
    #===================================================================================================
    # Tiến hành rút trích tin mật
    rs = bitarray.bitarray()
    for idxLine, line in enumerate(cover_text_lines):       # Duyệt từng dòng dữ liệu của văn bản phủ
        if(idxLine in lstNoneAlign):                        # Nếu dòng dữ liệu không phải dòng căn lề thì xét đến dòng tiếp theo
            continue
        # Lấy danh sách các khoảng trắng của dòng
        lstSpace = findAllSpaceInLine(line)
        
        # Loại bỏ index của khoảng trắng thứ 2 trong của vùng 2 khoảng trắng
        for idxRemove, space in enumerate(lstSpace, start = 1): 
            if(idxRemove >= len(lstSpace)):
                break
            if(space + 1 == lstSpace[idxRemove]):
                del lstSpace[idxRemove]

        #Rút trích bit
        for idxSpace in range(0, len(lstSpace) - 1, 2):            
            idxChar = lstSpace[idxSpace]
            idxCharNext1 = lstSpace[idxSpace + 1]

            if(line[idxChar : idxChar + 1] == ' ' and line[idxChar + 1 : idxChar + 2] == ' ' 
               and line[idxCharNext1 : idxCharNext1 + 1] == ' '  and line[idxCharNext1 + 1 : idxCharNext1 + 2] != ' '):       # 2 khoảng trắng rồi 1 khoảng trắng => bit 0
                if(idxChar + 2 == idxCharNext1):
                    continue
                rs.append(False)
            elif(line[idxChar : idxChar + 1] == ' ' and line[idxChar + 1 : idxChar + 2] != ' ' 
                 and line[idxCharNext1 : idxCharNext1 + 1] == ' ' and line[idxCharNext1 + 1 : idxCharNext1 + 2] == ' '):     # 1 khoảng trắng rồi 2 khoảng trắng
                rs.append(True)
    
    
    extrTemp = rs[::-1]
    for idxSplit in range(0, len(extrTemp) - 4, 1):
        if(extrTemp[idxSplit]):
            extrTemp = extrTemp[idxSplit + 1:]
            break
    
    # Ghi kết ra file stego.txt
    with open(extr_msg_file, 'w') as f:
        f.write(extrTemp[::-1].tostring())
    return True

In [9]:
# TEST
extract('correct_stego.txt', 'extr_msg.txt')
with open('extr_msg.txt', 'r') as f:
    extr_msg = f.read()
with open('correct_extr_msg.txt', 'r') as f:
    correct_extr_msg = f.read()
assert extr_msg == correct_extr_msg